In [1]:
import pickle
import boto3
import os

In [2]:
# Initializing S3 access
key_id = os.environ.get("AWS_ACCESS_KEY_ID")
secret_key = os.environ.get("AWS_SECRET_ACCESS_KEY")
endpoint = os.environ.get("AWS_S3_ENDPOINT")
bucket_name = os.environ.get("AWS_S3_BUCKET")
xgboost_model_bucket_key = "gradient_boost_model"
rf_model_bucket_key = "rf_model"
best_model_bucket_key = "best_model"

session = boto3.session.Session(aws_access_key_id=key_id, aws_secret_access_key=secret_key)
s3 = session.resource(service_name='s3', endpoint_url=endpoint)

In [ ]:
# Load the xgboost model
xgb_model_pickle = s3.Bucket(bucket_name).Object(xgboost_model_bucket_key).get()['Body'].read()
(xgb_model, xgb_accuracy) = pickle.loads(xgb_model_pickle)

In [ ]:
# Load the randomforest model
rf_model_pickle = s3.Bucket(bucket_name).Object(rf_model_bucket_key).get()['Body'].read()
(rf_model, rf_accuracy) = pickle.loads(rf_model_pickle)

In [ ]:
# Compare and upload the best one
if xgb_accuracy>rf_accuracy:
    model_and_metadata = {
        "model":xgb_model,
        "model_type":"xgboost",
        "accuarcy":xgb_accuracy
    }
    pickled_model_and_metadata = pickle.dumps(model_and_metadata, protocol=pickle.HIGHEST_PROTOCOL)
    s3.Bucket(bucket_name).put_object(Key=best_model_bucket_key, Body=pickled_model_and_metadata)
else:
    model_and_metadata = {
        "model":rf_model,
        "model_type":"randomforest",
        "accuarcy":rf_accuracy
    }
    pickled_model_and_metadata = pickle.dumps(model_and_metadata, protocol=pickle.HIGHEST_PROTOCOL)
    s3.Bucket(bucket_name).put_object(Key=best_model_bucket_key, Body=pickled_model_and_metadata)